### Data Cleaning

In [1]:
#### Cargue de las bases de datos

import pandas as pd 

customers = pd.read_csv('Customers.csv',encoding='latin1',delimiter=';')                     
location = pd.read_csv('Location.csv',encoding='latin1',delimiter=';')
orders = pd.read_csv('Orders.csv', encoding='latin1', delimiter=';')
products=pd.read_csv('Products.csv',encoding='latin1',delimiter=';')

In [2]:
# ...existing code...
print(orders.dtypes)
# ...existing code...

Row ID          int64
Order ID       object
Order Date     object
Ship Date      object
Ship Mode      object
Customer ID    object
Segment        object
Postal Code     int64
Product ID     object
Sales          object
Quantity        int64
Discount       object
Profit         object
dtype: object


### Creación de Indicadores

1. KPI´s Total Sales

In [3]:
#...existing code...
orders['Sales'] = orders['Sales'].str.replace(',', '.', regex=False)
orders['Sales'] = pd.to_numeric(orders['Sales'], errors='coerce')
# ...existing code...
# ...existing code...
orders['Order Date'] = pd.to_datetime(orders['Order Date'], dayfirst=True, errors='coerce')
orders['Year'] = orders['Order Date'].dt.year
# ...existing code...

In [4]:
import plotly.graph_objects as go

def tarjeta_total_sales(df, año_seleccionado):
    ventas_por_año = df.groupby('Year')['Sales'].sum()
    ventas_actual = ventas_por_año.get(año_seleccionado, 0)
    ventas_anterior = ventas_por_año.get(año_seleccionado - 1, 0)
    if ventas_anterior != 0:
        diff_pct = ((ventas_actual - ventas_anterior) / ventas_anterior) * 100
    else:
        diff_pct = 0

    fig = go.Figure(go.Indicator(
        mode = "number+delta",
        value = ventas_actual,
        number = {'valueformat': ',.0f', 'prefix': '$'},
        delta = {'reference': ventas_anterior, 'relative': True, 'valueformat': '.2%', 'position': "bottom"},
        title = {"text": f"<b>Total Sales</b><br><span style='font-size:0.8em;color:gray'>{año_seleccionado}</span>"}
    ))
    fig.update_layout(height=250)
    fig.show()

# Ejemplo de uso:
tarjeta_total_sales(orders, 2022)

In [8]:
orders['Order Date'] = pd.to_datetime(orders['Order Date'])  # convertir a fecha si no lo está
orders['Month'] = orders['Order Date'].dt.month  # extraer el mes


In [7]:
orders.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Segment', 'Postal Code', 'Product ID', 'Sales',
       'Quantity', 'Discount', 'Profit', 'Year'],
      dtype='object')

In [13]:
import plotly.graph_objects as go
import pandas as pd

# Asegúrate de tener la columna 'Month'
orders['Order Date'] = pd.to_datetime(orders['Order Date'])
orders['Month'] = orders['Order Date'].dt.month

años_disponibles = sorted(orders['Year'].unique())
fig = go.Figure()
trazas = []

# Creamos las trazas de cada año
for i, año in enumerate(años_disponibles):
    actual = orders[orders['Year'] == año].groupby('Month')['Sales'].mean()
    anterior = orders[orders['Year'] == año - 1].groupby('Month')['Sales'].mean()

    total_actual = orders[orders['Year'] == año]['Sales'].sum()
    total_anterior = orders[orders['Year'] == año - 1]['Sales'].sum() if not anterior.empty else 0
    diff_pct = ((total_actual - total_anterior) / total_anterior * 100) if total_anterior else 0

    max_mes = actual.idxmax()
    min_mes = actual.idxmin()

    # Línea gris: año anterior
    trazas.append(go.Scatter(
        x=anterior.index,
        y=anterior.values,
        mode='lines',
        name=f"{año - 1}",
        line=dict(color='lightgray', width=2),
        visible=(i == len(años_disponibles) - 1)
    ))

    # Línea negra: año actual
    trazas.append(go.Scatter(
        x=actual.index,
        y=actual.values,
        mode='lines',
        name=f"{año}",
        line=dict(color='black', width=3),
        visible=(i == len(años_disponibles) - 1)
    ))

    # Punto mínimo (naranja)
    trazas.append(go.Scatter(
        x=[min_mes],
        y=[actual[min_mes]],
        mode='markers',
        marker=dict(color='orange', size=12),
        name='Min',
        visible=(i == len(años_disponibles) - 1)
    ))

    # Punto máximo (azul)
    trazas.append(go.Scatter(
        x=[max_mes],
        y=[actual[max_mes]],
        mode='markers',
        marker=dict(color='dodgerblue', size=12),
        name='Max',
        visible=(i == len(años_disponibles) - 1)
    ))

# Añadir todas las trazas al gráfico
for traza in trazas:
    fig.add_trace(traza)

# Crear botones del dropdown por año
botones = []
for i, año in enumerate(años_disponibles):
    actual = orders[orders['Year'] == año].groupby('Month')['Sales'].mean()
    anterior = orders[orders['Year'] == año - 1].groupby('Month')['Sales'].mean()
    total_actual = orders[orders['Year'] == año]['Sales'].sum()
    total_anterior = orders[orders['Year'] == año - 1]['Sales'].sum() if not anterior.empty else 0
    diff_pct = ((total_actual - total_anterior) / total_anterior * 100) if total_anterior else 0

    visibilidad = [False] * len(trazas)
    visibilidad[i*4:i*4+4] = [True] * 4

    botones.append(dict(
        label=str(año),
        method='update',
        args=[
            {'visible': visibilidad},
            {'annotations': [
                dict(x=0.01, y=1.15, xref='paper', yref='paper',
                     text=f"<b>Avg. Monthly Sales</b><br><span style='font-size:1.5em'>$ {total_actual:,.0f}</span>",
                     showarrow=False, align='left'),
                dict(x=0.01, y=1.05, xref='paper', yref='paper',
                     text=f"<span style='color:gray;'>▲ {diff_pct:.1f}% vs. PY</span>",
                     showarrow=False, align='left')
            ]}
        ]
    ))

# Valores por defecto (último año)
último_año = años_disponibles[-1]
df_ult = orders[orders['Year'] == último_año]['Sales']
df_ant = orders[orders['Year'] == último_año - 1]['Sales']
ventas_ult = df_ult.sum()
ventas_ant = df_ant.sum()
diff_ult = ((ventas_ult - ventas_ant) / ventas_ant * 100) if ventas_ant.sum() else 0

# Layout y diseño
fig.update_layout(
    updatemenus=[dict(
        active=len(botones) - 1,
        buttons=botones,
        direction="down",
        x=0,
        y=1.25,
        showactive=True
    )],
    annotations=[
        dict(x=0.01, y=1.15, xref='paper', yref='paper',
             text=f"<b>Avg. Monthly Sales</b><br><span style='font-size:1.5em'>$ {ventas_ult:,.0f}</span>",
             showarrow=False, align='left'),
        dict(x=0.01, y=1.05, xref='paper', yref='paper',
             text=f"<span style='color:gray;'>▲ {diff_ult:.1f}% vs. PY</span>",
             showarrow=False, align='left')
    ],
    height=500,
    margin=dict(t=100, b=40),
    plot_bgcolor='white',
    xaxis=dict(title='Month', dtick=1),
    yaxis=dict(title='Average Sales'),
    showlegend=False
)

fig.show()
